---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [269]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from math import ceil
from itertools import product

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [270]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [271]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_table("university_towns.txt", sep = "/n", header = None)
    df.columns = ["RegionName"]
    df["State"] = None
    #Step1 : Assign States : States are at the top rows marked with [edit]
    state = None
    for (index, row) in df.iterrows():
        if (df.iloc[index]["RegionName"][-6:] == "[edit]"):
            state = df.iloc[index]["RegionName"][:-6]
        df.iloc[index]["State"] = state
    #Step2 : Exclude rows that are just states
    df = df[~df["RegionName"].str.contains("\[edit\]")]
    #Step3 : Remove () and [] and contained strings from city names
    df["RegionName"] = df['RegionName'].str.replace(r"\[.{0,}", "")
    df["RegionName"] = df['RegionName'].str.replace(r" \(.{0,}", "")
    #Step4 : Put columns in order required by question1
    new_order = ["State", "RegionName"]
    df = df.reindex_axis(new_order, axis =1)
    df.reset_index(inplace= True, drop = True)
    return df

In [272]:

get_list_of_university_towns()


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [273]:
skiprows = list(range(5))
skiprows.extend((6,7))

gdplev_df = pd.read_excel("gdplev.xls", skiprows=skiprows)

del(gdplev_df["Unnamed: 3"])

gdplev_df.rename(columns = {"Unnamed: 4": "YearQuater"}, inplace= True)
gdplev_df = gdplev_df[gdplev_df["YearQuater"] >= "2000q1"]

In [274]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    colname = "GDP in billions of current dollars.1"
    recessionstart = None
    for index in range (0, len(gdplev_df)-2):
        currentGDP = gdplev_df.iloc[index][colname] 
        next = gdplev_df.iloc[index+1][colname]
        next_to_next = gdplev_df.iloc[index+2][colname]
        if((currentGDP > next) & (next > next_to_next)):
            recessionstart = gdplev_df.iloc[index]["YearQuater"]
            break;
    
    return recessionstart

In [275]:
#len(gdplev_df)
#gdplev_df[:-61]
get_recession_start()

'2008q3'

In [276]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    colname = "GDP in billions of current dollars.1"
    recessionend = None
    #get when recession started and work on that dataset
    start = get_recession_start()
    working_set = gdplev_df[gdplev_df["YearQuater"]>= start]
    for index in range (0, len(working_set)-2):
        currentGDP = working_set.iloc[index][colname] 
        next = working_set.iloc[index+1][colname]
        next_to_next = working_set.iloc[index+2][colname]
        if((currentGDP < next) & (next < next_to_next)):
            recessionend = working_set.iloc[index+2]["YearQuater"]
            break;
       
    return recessionend

In [277]:
get_recession_end()

'2009q4'

In [278]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdpcolname = "GDP in billions of current dollars.1"
    #Step1 : Get when recession started
    start = get_recession_start()
    end =  get_recession_end()
    #Step2 : Get data in between and reset index so that we can work with idxmax and mins
    working_set = gdplev_df[gdplev_df["YearQuater"].between(start, end)]
    working_set.reset_index(drop = True, inplace = True)
    #Step3: Get year and quarter when it iwas minimum
    yearquarter = working_set.iloc[working_set[gdpcolname].idxmin()]["YearQuater"]
    return yearquarter

In [279]:
get_recession_bottom()


'2009q2'

In [280]:
def convert_housing_data_to_quarters():
    '''
    Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    #Step1: Read housing data
    housing_data = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    #Step2 : Rename the columns to format <Year><Quarter>
    rename = {}
    allcolumns = housing_data.columns
    for column in allcolumns:
        if(column[0:4].isdigit()):       
            year, month = column.split("-")
            month = int(month)
            quarter = "q"+ str(ceil(month/3))
            newname = str(year)+ str(quarter)
            rename[column]= newname
    housing_data.rename(columns = rename, inplace= True)
    
    #Step3 : Change State from 2 letter to full word using the dict provided.
    housing_data["State"] = housing_data["State"].map(states)
    
    # Generate required colum names
    requiredcolumns = []
    for item in product(list(range(2000, 2017)), ["q1", "q2", "q3", "q4"]):
        value = map(str, item)
        requiredcolumns.append ("".join(value))
    #Eliminate 2016q4 as required in the assignment
    requiredcolumns = requiredcolumns[:-1]
    
    housing_quarter_df = pd.DataFrame(columns= requiredcolumns)
    housing_quarter_df["State"] = housing_data["State"]
    housing_quarter_df["RegionName"] = housing_data["RegionName"]
    for column in requiredcolumns:
        housing_quarter_df[column] = housing_data[column].mean(axis =1)
    housing_quarter_df.set_index(["State", "RegionName"], inplace= True)
    
    return housing_quarter_df

In [281]:
df = convert_housing_data_to_quarters()

In [318]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    recession_start = get_recession_start() 
    recession_bottom = get_recession_bottom()
    univ_towns = get_list_of_university_towns()
    
    
    # get housing data 
    recession_df = convert_housing_data_to_quarters()
    recession_df.reset_index(inplace= True)
    #get housing data for recession start and bottom and get trend
    recession_df["loss"] = recession_df[recession_start]-recession_df[recession_bottom]
    
    
    
    temp = pd.merge(recession_df, univ_towns, on = ["State","RegionName"], how = "left", indicator="RecordPresentIn")
    recession_df["IsUnivTown"] = np.where(temp.RecordPresentIn == "both", 1,0)

    
    recession_unitowns_df = recession_df[recession_df["IsUnivTown"] ==1]["loss"].dropna()
    recession_nonunitowns_df = recession_df[recession_df["IsUnivTown"] ==0]["loss"].dropna()
    
    t, p = ttest_ind(recession_unitowns_df, recession_nonunitowns_df, nan_policy = "omit")
    
    different = True if (p<0.01) else False
    better = "university town" if (t<0) else "non-university town"
    result = different, p, better
    
    return result

In [319]:
run_ttest()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


(True, 0.0043252148535112009, 'university town')